# State Farm Distracted Driver Detection: Take 3
_Can computer vision spot distracted drivers?_

---

## Lesson 3 Homework Assignment

Dataset: https://www.kaggle.com/c/state-farm-distracted-driver-detection

### Dealing with Overfitting
In [`lesson2-hmwk.ipynb`](https://github.com/iconix/fast.ai/blob/master/nbs/lesson2-hmwk.ipynb), my final results (after 5 epochs) were as follows:
> `loss: 0.6260 - acc: 0.7907 - val_loss: 1.6719 - val_acc: 0.4978`

When `val_acc >> acc`, this is a clear indicator of overfitting on the training data.

On the bright side, this means that my neural net architecture is complex enough to model the data. The next step is to generalize my architecture a bit more.

Here is the prioritized list of approaches to reducing overfitting provided during class:
1. Add more data
2. Use data augmentation
3. Use architectures that generalize well
4. Add regularization (dropout, L2/L1 regularization)
5. Reduce architecture complexity

### Downloading and creating the datasets

This week, I am skipping the download, split, and create of training, validation, test, and sample datasets, relying instead on the data split from last week. See [`lesson2-hmwk.ipynb`](https://github.com/iconix/fast.ai/blob/master/nbs/lesson2-hmwk.ipynb) for those steps, if needed.

Additionally, I will be starting with the weights from last week below (`finetune2.h5`).

### Basic Configuration

In [2]:
import os

current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/statefarm/'

# point to your training images
train_dir = DATA_HOME_DIR + 'train'

# point to the 'driver_imgs_list.csv'
lookup = DATA_HOME_DIR + 'driver_imgs_list.csv'

# point to the validation directory, which will be created in the next block
val_dir = DATA_HOME_DIR + 'valid'

sample_dir = DATA_HOME_DIR + 'sample'

test_dir = DATA_HOME_DIR + 'test'

#path = DATA_HOME_DIR + 'sample/'
path = DATA_HOME_DIR
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [6]:
%matplotlib inline

In [5]:
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [7]:
batch_size=64

### Load pre-trained model layers

In [8]:
model = vgg_ft(10)

In [29]:
finetune2_path = model_path+'finetune2.h5'

model.load_weights(finetune2_path);

In [30]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [9]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

conv_layers = layers[:last_conv_idx+1]
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

### Data Augmentation

Define a generator that includes data augmentation (convenient feature of Keras).

In [32]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True)

In [33]:
trn_batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 18298 images belonging to 10 classes.
Found 4126 images belonging to 10 classes.


Jeremy's [explanation](http://forums.fast.ai/t/lesson-3-discussion/186/33) as to why we aren't training the convolutional layers here:
> The early layers are so general (e.g. remember Zeiler's visualizations - layer 1 just finds edges and gradients) that it's extremely unlikely that you'll need to change them, unless you're looking at very different kinds of images. e.g. if you're classifying line drawings, instead of photos, you'll probably need to retrain many conv layers too.

In [10]:
for layer in layers[:last_conv_idx+1]: layer.trainable=False

In [11]:
# Updating slowly because it is finely tuned
K.set_value(model.optimizer.lr, 0.00001)

In [37]:
model.fit_generator(trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/8
18298/18298 [==============================] - 590s - loss: 2.3606 - acc: 0.4514 - val_loss: 1.6972 - val_acc: 0.5264
Epoch 2/8
18298/18298 [==============================] - 590s - loss: 2.2040 - acc: 0.4697 - val_loss: 1.7198 - val_acc: 0.5201
Epoch 3/8
18298/18298 [==============================] - 590s - loss: 2.1467 - acc: 0.4716 - val_loss: 1.7177 - val_acc: 0.5208
Epoch 4/8
18298/18298 [==============================] - 591s - loss: 2.1106 - acc: 0.4792 - val_loss: 1.7157 - val_acc: 0.5223
Epoch 5/8
18298/18298 [==============================] - 591s - loss: 2.0800 - acc: 0.4773 - val_loss: 1.7235 - val_acc: 0.5182
Epoch 6/8
18298/18298 [==============================] - 590s - loss: 2.0187 - acc: 0.4837 - val_loss: 1.7172 - val_acc: 0.5182
Epoch 7/8
18298/18298 [==============================] - 590s - loss: 1.9916 - acc: 0.4862 - val_loss: 1.7573 - val_acc: 0.5107
Epoch 8/8
18298/18298 [==============================] - 590s - loss: 1.9278 - acc: 0.5002 - val_loss: 1

After 8 epochs, we are overfitting much much less, which is great!
> `loss: 1.9278 - acc: 0.5002 - val_loss: 1.7463 - val_acc: 0.5099`

In fact, now we're underfitting very slightly. This seems like a good time to try decreasing dropout a smidge, just to see what happens.

Let's save our intermediate weights first, just in case.

In [12]:
finetune_hw3_1_path = model_path+'finetune_hw3_1.h5'
if not os.path.exists(finetune_hw3_1_path):
    model.save_weights(finetune_hw3_1_path)
model.load_weights(finetune_hw3_1_path)

#### Intermediate Kaggle submission

Interestingly, despite the significant decrease in overfitting, `val_acc` only improved a small amount from `0.4978`... I'd like to try submitting these results to Kaggle, just to see how this result reflects in the rankings.

In [13]:
from IPython.display import FileLink

subm_name = 'subm_hmwk3_1.gz'
subm_path = path + 'results/' + subm_name

#Get the classes from the validation batch
val_preprocess = get_batches(path+'valid', shuffle=False, batch_size=1)
classes = sorted(val_preprocess.class_indices, key=val_preprocess.class_indices.get)

Found 4126 images belonging to 10 classes.


In [ ]:
test_data = get_data(path+'test')
preds = model.predict(test_data, batch_size = batch_size*2)

Found 79726 images belonging to 1 classes.


In [ ]:
# (number of classes - 1) = 9 (http://forums.fast.ai/t/moving-up-the-ncfm-leaderboard-by-100-positions-do-clip/1035/6)
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
subm = do_clip(preds, 0.51)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')
FileLink(subm_name)

### Increase Dropout
We're going to experiment with increasing the dropout rate from Vgg16's 50%.

In [ ]:
conv_model = Sequential(conv_layers)
fc_model = Sequential(fc_layers)

def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model